# Fetching Tweets in Real Time using Twitter API

Tweepy library is used to route all the Live tweets to a TCP socket server.<br> 
This server is hosted in same local machine.

**Import tweepy library**

In [1]:
! pip3 install tweepy==3.8.0 # latest version of Tweepy does not have StreamListener

Defaulting to user installation because normal site-packages is not writeable
  Using cached tweepy-3.8.0-py2.py3-none-any.whl (28 kB)


In [2]:
import tweepy
print(tweepy.__version__)

3.8.0


**Import OAuthHandler**<br> Used for Authentication protocol (OAuth) of Twitter API.

In [3]:
from tweepy import OAuthHandler

**Import Stream & StreamListner**<br> Listen to streaming data getting from the socket.

In [4]:
from tweepy import Stream

In [6]:
from tweepy.streaming import StreamListener

**Import socket** <br> It is used to create socket connection with Localhost i.e. local server. <br>
**Import Json** <br>Json is used because twitter data comes in json format.

In [7]:
import socket
import json

To connect to Twitter API, we need these 4 Keys. <br>
It can be generated from https://apps.twitter.com/ by creating a new app.

In [8]:
consumer_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
consumer_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
access_token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
access_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"

#### Class TweetsListner
    1.listening to the stream via StreamListener. 
    2.constrctor function (init) : initializing the socket. 
    3.tweet_data contains the tweet's json coming from the stream. 
    4.In the tweet's json object 'text' field contains the actual Tweet.
    5.The actual tweet is extracted & sent to the client socket.
    6.on_error is used to catch & throw errors.

In [9]:
class TweetsListener(StreamListener):
    def __init__(self, csocket):
        self.client_socket = csocket
    
    def on_data(self, tweet_json):
          try:
                tweet_data = json.loads( tweet_json )
                print("tweet_text: ", tweet_data['text'].encode('utf-8') )
                print("created_at: ", tweet_data['created_at'])  
                print("name: ", tweet_data['user']['name'])              
                print("location: ", tweet_data['user']['location'])
                print("\n")
                s = tweet_data['text'] + "\n"
                print(s)
                s = s.encode('utf-8')
                self.client_socket.send( s )
                return True
          
          except BaseException as e:
                print("Error on_data: %s" % str(e))
                return True
    def on_error(self, status):
        print(status)
        return True

### Function send_twitter_data():
    1. auth is doing Authentication check using OAuthHandler. It uses 4 keys of Twitter APIs.
    2. consumer_key & consumer_secret are like username & access_token & access_secret are like password.
    3. twitter_stream will get the actual twitter live stream data.
    4. It'll call Stream with auth & call the Class TweetsListner & pass c_socket info to the class.
    5. From the stream of tweets, filter & take only tweets which contains "corona" word.

In [10]:
def send_twitter_data(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  twitter_stream_data = Stream(auth, TweetsListener(c_socket))
  twitter_stream_data.filter(track=['corona'])

### Socket Creation:
    1. Socket is created by using socket function.
    2. Host is localhost, which is machine IP, where this Jupyter notebook is running.
    3. Port is 7773 (It can be anything, unique for this service)
    4. Now bind host & port address to the socket.

In [11]:
# Create a socket object
s = socket.socket()        
# s.TCPServer.allow_reuse_address = True

# Reserve a port for your service.
port = 7777

# Bind to the port
s.bind(('127.0.0.1', port))        
print ("socket binded to %s" %(port))

# print
s.listen(5)    
print ("socket is listening") 

socket binded to 7777
socket is listening


socket will wait & listen for few seconds. <br>
After that connection is made.

In [12]:
# Now wait for client connection.
c, addr = s.accept()

In [13]:
# Establish connection with client.


### Send live Tweets in real time to the Socket
Function send_twitter_data will send all the Tweets containing word 'corona' to socket 'c' in real time.<br>
This will be consumed in the next code of Spark Streaming to do Trend Analysis.

In [14]:
send_twitter_data(c)

tweet_text:  b'@Cookiechip3 @schwaeblevonbc Die L\xc3\xbcge von der \xc3\x9cberlastung der Intensivstationen wegen Corona wurde deutlich aufgede\xe2\x80\xa6 https://t.co/kwx9t7yOD3'
created_at:  Tue Oct 26 15:18:03 +0000 2021
name:  Diese Wahrheit
location:  None


@Cookiechip3 @schwaeblevonbc Die Lüge von der Überlastung der Intensivstationen wegen Corona wurde deutlich aufgede… https://t.co/kwx9t7yOD3

tweet_text:  b'RT @CosmicSoulPoet: Corona means crown...\nas in Crown Chakra. \nIn Ocul at ion \xf0\x9f\x92\x89 means \nInto the Eye \xf0\x9f\x91\x81\n(Third eye blind)\nStay Sovereign. htt\xe2\x80\xa6'
created_at:  Tue Oct 26 15:18:03 +0000 2021
name:  ᴛʜᴇ ꜱʜᴀᴡᴍᴀɴ's ᴅᴀᴜɢʜᴛᴇʀ
location:  where the wild may roam


RT @CosmicSoulPoet: Corona means crown...
as in Crown Chakra. 
In Ocul at ion 💉 means 
Into the Eye 👁
(Third eye blind)
Stay Sovereign. htt…

tweet_text:  b'@jensspahn \nhttps://t.co/nTTulkZtSW'
created_at:  Tue Oct 26 15:18:04 +0000 2021
name:  Steffi Marburger
location:  NRW


KeyboardInterrupt: 